In [ ]:
#import bibliothèques
from sklearn import preprocessing
from joblib import dump, load

from datetime import datetime
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, Dense, Dropout #Pour instancier une couche Dense et une d'Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# chargements des jeux de données d'entrainement et et de test
pd.set_option('display.max_columns', None)

data_path = '../../Data'
models_path = '../../Models'

X_train = pd.read_csv('{0}/X_train.zip'.format(data_path), compression=dict(method='zip', archive_name='X_train.csv'))
X_test = pd.read_csv('{0}/X_test.zip'.format(data_path), compression=dict(method='zip', archive_name='X_test.csv'))
y_train = pd.read_csv('{0}/y_train.zip'.format(data_path), compression=dict(method='zip', archive_name='y_train.csv'))
y_test = pd.read_csv('{0}/y_test.zip'.format(data_path), compression=dict(method='zip', archive_name='y_test.csv'))

y_train = y_train['grav']
y_test = y_test['grav']



In [ ]:
#binarisation de la cible
y_train  = (y_train>=2) *1
y_test = (y_test>=2) * 1

In [ ]:
us = RandomUnderSampler()
X_train, y_train = us.fit_resample(X_train, y_train)

Réseau de neurones denses

In [ ]:
y_test = to_categorical(y_test, num_classes=4)
y_train= to_categorical(y_train, num_classes=4)

In [ ]:
model = Sequential()

model.add(Dense(units = 256, activation = 'relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(rate=.3))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(rate=.3))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate=.3))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate=.3))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dropout(rate=.3))
model.add(Dense(units = 4, activation = 'softmax'))


model.summary()

In [ ]:
m = Recall(class_id = 1, name = 'recall_class_1')


In [ ]:
opt = Adam(learning_rate=0.01)
model.compile(loss = "categorical_crossentropy",
              optimizer = opt,
              metrics = ['accuracy',m])



In [12]:
model.fit(X_train, y_train, epochs = 500, batch_size = 32, validation_split = .1)


Epoch 1/500
9968/9968 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.8430 - loss: 0.3594 - recall: 0.3697 - val_accuracy: 0.8516 - val_loss: 0.3323 - val_recall: 0.3963
Epoch 2/500
9968/9968 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.8524 - loss: 0.3285 - recall: 0.4191 - val_accuracy: 0.8547 - val_loss: 0.3270 - val_recall: 0.4280
Epoch 3/500
9968/9968 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.8546 - loss: 0.3227 - recall: 0.4402 - val_accuracy: 0.8545 - val_loss: 0.3254 - val_recall: 0.4081
Epoch 4/500
9968/9968 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.8567 - loss: 0.3193 - recall: 0.4425 - val_accuracy: 0.8560 - val_loss: 0.3241 - val_recall: 0.4289
Epoch 5/500
9968/9968 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.8556 - loss: 0.3200 - recall: 0.4507 - val_accuracy: 0.8557 - val_loss: 0.3240 - val_recall: 0.3982
Epoch 6/500
9968/9968 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.8570 - loss: 0.3175 - recall: 0.4544 - val_accuracy: 0.8553 - val_loss: 0.3224 -

Estimation

In [14]:
y_pred = model.predict(X_test)

#y_pred_class = np.argmax(y_pred,axis=1)
#y_test_class = 0 + y_test[1] * 1 + y_test[2]*2 + y_test[3]*3

2769/2769 ━━━━━━━━━━━━━━━━━━━━ 3s 912us/step


In [ ]:
#y_test_class = 0 + y_test[:,1] * 1 + y_test[:,2]*2 + y_test[:,3]*3


In [15]:
dump(model, './modèles/dnn_2_classes.joblib')






['./modèles/dnn_2_classes.joblib']

In [19]:
y_pred = (y_pred > 0.5)*1

In [24]:
y_pred = y_pred[:,0]

In [25]:
print('####   Matrice de confusion')
print(pd.crosstab(y_test, y_pred, rownames=['Gravité'], colnames=['Prédiction']))
print('####   Matrice de confusion normalisée')
pd.crosstab(y_test, y_pred, rownames=['Gravité'], colnames=['Prédiction'], normalize='index')
print('####   Rapport de classification')
print(classification_report_imbalanced(y_test, y_pred))

####   Matrice de confusion
Prédiction      0     1
Gravité                
0           67505  4882
1            8414  7797
####   Matrice de confusion normalisée
####   Rapport de classification
                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.93      0.48      0.91      0.67      0.47     72387
          1       0.61      0.48      0.93      0.54      0.67      0.43     16211

avg / total       0.84      0.85      0.56      0.84      0.67      0.46     88598



In [23]:
y_pred[:,0]

array([1, 1, 0, ..., 0, 0, 0])

In [ ]:
y_test

: 

: 